In [1]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from components import Components
import os
load_dotenv()

e:\Github Repositories\RAG-FYP2024\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

#### Setting up OpenAI GPT4 Mini

In [2]:
token = os.getenv("GITHUB_TOKEN")
route = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini"


In [3]:
client = OpenAI(
    base_url=route,
    api_key=token,
)

In [4]:
def get_response(prompt):
    response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """Instructions:
                        - Be helpful and answer questions concisely. If you don't know the answer, say 'I don't know'
                        - Be concise and to the point.""",
        },
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=model_name,
    temperature=1.,
    max_tokens=1000,
    top_p=1.
)
    return response.choices[0].message.content

#### Setting up GroqLLM


In [2]:
rag_components = Components("Snowflake/snowflake-arctic-embed-s","mixedbread-ai/mxbai-embed-large-v1","gemma2-9b-it")

In [3]:
llm = rag_components.get_groq_llm(system_prompt=""" Instructions:
        - Try your hardest to answer the question correctly.
        - Be concise and to the point.""")

#### Loading Data

In [4]:
df_qa = pd.read_csv('../data/MainDataset/data/QA.csv')

In [5]:
question_list = []
for questions in df_qa['question']:
    question_list.append(questions)

In [6]:
import time
response_list = []
for i in range(len(question_list)): 
    time.sleep(3)
    print(question_list[i])
    #response = get_response(question_list[i])
    response = llm.complete(question_list[i])
    print(response)
    rowIndex = df_qa.index[i]
    #response_list.append(response)
    df_qa.at[rowIndex, 'gen_answer'] = response.text
    print('')
    print("--"*100)
    print(f'{i}/{128}')
 


who introduced modern methods of surgery, such as antiseptics, sterilization, and washing hands? brainly
**Joseph Lister** is credited with introducing modern methods of surgery, including antiseptics, sterilization, and handwashing. 


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0/128
what kind of government was chiang kai-shek trying to build in china
Chiang Kai-shek was trying to build a **modern, authoritarian republic** in China. 


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
1/128
president adams
Please clarify your question. What about President Adams would you like to know? 

For example, you could ask:

* What were President Adams's main accomplishments?
* What were Pr

In [9]:
df_qa.to_csv(f'../data/MainDataset/results/{rag_components.model_name}_Run1_BaseLLMNORAG.csv',index=False)